# AOC 2021

Welcome to the Advent of Code 2021 !

## Basic configuration



In [ ]:
!pip install aocd

In [ ]:
import os

os.environ['AOC_SESSION'] = open('session.txt').read().strip()

In [ ]:
from aocd import submit
from aocd.models import Puzzle

## Day 25
https://adventofcode.com/2021/day/25
### Part 1

In [ ]:
puzzle = Puzzle(2021, 25)

In [ ]:
lines = puzzle.input_data.split('\n')

In [ ]:
import numpy as np

from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (10,10)

height, width = len(lines), len(lines[0])

cmap = {
    '.': 0,
    '>': 1,
    'v': 2,
}

def parse_input(lines):
    floor = np.zeros((width, height))
    for y, line in enumerate(lines):
        for x, char in enumerate(line):
            floor[x,y] = cmap[char]
    return floor

In [ ]:

def step(floor):
    
    for v in [1, 2]:
        new_floor = np.zeros((width, height))

        for y in range(height):
            for x in range(width):
                if floor[x, y] == v:
                    xn, yn = x, y
                    if v == 1 and (floor[(x+1)%width, y] == 0):
                        xn = (x+1)%width
                    if v == 2 and (floor[x, (y+1)%height] == 0):
                        yn = (y+1)%height
                    new_floor[xn, yn] = v
                    
        new_floor[np.where(floor == v%2+1)] = v%2+1
        floor = new_floor
    return new_floor

In [ ]:
count = 0

floor = parse_input(lines)

while True:
    new_floor = step(floor)
    count += 1
    if np.all(new_floor == floor):
        break
    floor = new_floor

In [ ]:
plt.imshow(floor)

In [ ]:
answer_a = count

In [ ]:
puzzle.answer_a = answer_a

## Day 24
https://adventofcode.com/2021/day/24
### Part 1

### Part 2

## Day 23
https://adventofcode.com/2021/day/23
### Part 1

In [ ]:
hallway = [(x, 0) for x in range(11)]
valid_hallway = [(x, 0) for x in range(11) if x not in [2, 4, 6, 8]]

weights = {cls: 10**k for cls, k in zip("ABCD", range(4))}

In [ ]:
lair = {class_: set((x, y) for y in range(1, 3)) for class_, x in zip("ABCD", [2, 4, 6, 8])}
lairs = [(x, y) for x in [2, 4, 6, 8] for y in range(1, 3)]
positions = hallway + lairs

idx2pos = {idx: pos for idx, pos in enumerate(positions)}
pos2idx = {pos: idx for idx, pos in enumerate(positions)}

In [ ]:
from collections import defaultdict


def get_transitions():

    matrix = {}

    for class_ in "ABCD":
        matrix[class_] = defaultdict(set)
        for src in lairs:
            for dst in lair[class_]:
                if src not in lair[class_] or (src in lair[class_] and src[1] < dst[1]):
                    matrix[class_][src].add(dst)
            for dst in valid_hallway:
                matrix[class_][src].add(dst)

        for src in valid_hallway:
            for dst in lair[class_]:
                matrix[class_][src].add(dst)
                
    return matrix


def get_path(src, dst):
    x1, y1 = src
    x2, y2 = dst
        
    path = set()

    while y1 != 0:
        path.add((x1, y1))
        y1 -= 1
    dx = 1 if x2 > x1 else -1
    while x1 != x2:
        path.add((x1, y1))
        x1 += dx
    while y1 != y2:
        path.add((x1, y1))
        y1 += 1
        
    path.remove(src)
    path.add(dst)
        
    return path

def display(state, large=False):
    
    state_dict = todict(state)
    
    for y in range(5 if large else 3):
        for x in range(11):
            if (x, y) in state_dict:
                print(state_dict[(x, y)], end="")
            else:
                print('#', end="")
        print()

def update_state(state, src, dst):
    idx_dst = pos2idx[dst]
    idx_src = pos2idx[src]
    new_state = state[:idx_dst] + state[idx_src] + state[idx_dst+1:]
    new_state = new_state[:idx_src] + '.' + new_state[idx_src+1:]
    return new_state

def has_foreigner(state, cls):
    return any( (state[pos2idx[pos]] != cls) and (state[pos2idx[pos]] != '.') for pos in lair[cls])

def todict(state):
    return {pos: state[idx] for idx, pos in enumerate(positions)}


In [ ]:
from collections import deque

def run(initial_state, final_state, MAX_COST = 20000, verbose = False):

    solutions = []
    visited = {initial_state: 0}
    queue = deque([(initial_state, [0], [initial_state])])
    
    transitions = get_transitions()

    while queue:
        state, cost, history = queue.popleft()

        pods = [(pos, state[idx]) for idx, pos in enumerate(positions) if state[idx] != '.']
        occupied = set(p[0] for p in pods)

        for src_pos, cls in pods:
            for dst_pos in transitions[cls][src_pos] - occupied:
                if src_pos in lair[cls] and not has_foreigner(state, cls) and dst_pos not in lair[cls]:
                    continue
                path = get_path(src_pos, dst_pos)

                if not (path & occupied):

                    if dst_pos in lair[cls] and has_foreigner(state, cls):
                        continue

                    path_cost = len(path)*weights[cls]

                    if sum(cost) + path_cost >= MAX_COST:
                        continue

                    new_state = update_state(state, src_pos, dst_pos)

                    if new_state == final_state:
                        if verbose:
                            print("="*40)
                            print(sum(cost))
                            print("="*40)
                            for c, h in zip(cost, history):
                                print(c)
                                display(h)
                                print('-'*20)
                        solutions.append(sum(cost) + path_cost)
                    else:
                        if new_state not in visited or (sum(cost) + path_cost) < visited[new_state]:
                            visited[new_state] = sum(cost) + path_cost
                            queue.append((new_state, cost + [path_cost], history + [new_state]))
    return solutions

In [ ]:
initial_state = "."*11 + "CBDAADBC"
display(initial_state)

final_state = "."*11 + "AABBCCDD"
display(final_state)

In [ ]:
solutions = run(initial_state, final_state)

In [ ]:
answer_a = min(solutions)

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
lair = {class_: set((x, y) for y in range(1, 5)) for class_, x in zip("ABCD", [2, 4, 6, 8])}
lairs = [(x, y) for x in [2, 4, 6, 8] for y in range(1, 5)]
positions = hallway + lairs

idx2pos = {idx: pos for idx, pos in enumerate(positions)}
pos2idx = {pos: idx for idx, pos in enumerate(positions)}

In [ ]:
initial_state = "."*11 + "CDDBDCBAABADBACC"
display(initial_state, large=True)

final_state = "."*11 + "AAAABBBBCCCCDDDD"
display(final_state, large=True)

In [ ]:
solutions = run(initial_state, final_state, MAX_COST=100000)

In [ ]:
answer_b = min(solutions)

In [ ]:
puzzle.answer_b = answer_b

## Day 22
https://adventofcode.com/2021/day/22
### Part 1

In [ ]:
puzzle = Puzzle(2021, 22)

In [ ]:
lines = puzzle.input_data.split('\n')

In [ ]:
lines = [line.split() for line in lines]

In [ ]:
from collections import deque

def read_input(lines, limit_scale=True):
    queue = deque()

    for cmd, coords in lines:
        x, y, z = coords.split(',')
        l = (x[2:].split('..'), y[2:].split('..'), z[2:].split('..'))
        if abs(int(l[0][0])) <= 50:
            limits = [(l[0][0], int(l[0][1])+1), (l[1][0], int(l[1][1])+1), (l[2][0], int(l[2][1])+1)]
            queue.append((cmd, Volume(limits)))
            
    return queue

In [ ]:
class Volume:
    def __init__(self, limits):
        self.x0 = int(limits[0][0])
        self.x1 = int(limits[0][1])
        self.y0 = int(limits[1][0])
        self.y1 = int(limits[1][1])
        self.z0 = int(limits[2][0])
        self.z1 = int(limits[2][1])
        
    def volume(self):
        return (self.x1 - self.x0)*(self.y1 - self.y0)*(self.z1 - self.z0)
    
    def intersect(self, other):
        x0 = max(self.x0, other.x0)
        x1 = min(self.x1, other.x1)
        y0 = max(self.y0, other.y0)
        y1 = min(self.y1, other.y1)
        z0 = max(self.z0, other.z0)
        z1 = min(self.z1, other.z1)
        
        if x0 >= x1 or y0 >= y1 or z0 >= z1:
            return None
        
        return Volume([(x0, x1), (y0, y1), (z0, z1)])
    
    def __repr__(self):
        return f"({self.x0},{self.x1}) ({self.y0},{self.y1}) ({self.z0},{self.z1})"
    
    def corners(self):
        corners = []
        for x in [self.x0, self.x1]:
            for y in [self.y0, self.y1]:
                for z in [self.z0, self.z1]:
                    corners.append((x, y, z))
        return corners
    
    def contains(self, pt, strict=True):
        x, y, z = pt
        if strict:
            return (self.x0 < x < self.x1) and (self.y0 < y < self.y1) and (self.z0 < z < self.z1)
        else:
            return (self.x0 <= x <= self.x1) and (self.y0 <= y <= self.y1) and (self.z0 <= z <= self.z1)

    def includes(self, other):
        return all(self.contains(pt, False) for pt in other.corners())

def linf(p1, p2):
    return sum(p1[i] != p2[i] for i in range(3))

def split(vol, inter):
    case = len(set(inter.corners()) & set(vol.corners()))
    subvolumes = []
    
    if case == 0:
        if inter.volume() < vol.volume():
            p0 = (vol.x0, vol.y0, vol.z0)
            p1 = (vol.x1, inter.y0, vol.z1)
            subvolumes.append(Volume(corners_to_limits(p0, p1)))
            
            p0 = (vol.x0, inter.y0, vol.z0)
            p1 = (vol.x1, inter.y1, inter.z0)
            subvolumes.append(Volume(corners_to_limits(p0, p1)))
            
            p0 = (vol.x0, inter.y0, inter.z1)
            p1 = (vol.x1, inter.y1, vol.z1)
            subvolumes.append(Volume(corners_to_limits(p0, p1)))
            
            p0 = (vol.x0, inter.y0, inter.z0)
            p1 = (inter.x0, inter.y1, inter.z1)
            subvolumes.append(Volume(corners_to_limits(p0, p1)))
            
            p0 = (inter.x1, inter.y0, inter.z0)
            p1 = (vol.x1, inter.y1, inter.z1)
            subvolumes.append(Volume(corners_to_limits(p0, p1)))
            
            p0 = (vol.x0, inter.y1, vol.z0)
            p1 = (vol.x1, vol.y1, vol.z1)
            subvolumes.append(Volume(corners_to_limits(p0, p1)))
    
    if case == 1:
        pivot = [pt for pt in inter.corners() if vol.contains(pt)][0]
        for corner in vol.corners():
            if corner not in set(inter.corners()):
                subvolumes.append(Volume(corners_to_limits(pivot, corner)))
    elif case == 2:
        anchor = list(set(vol.corners()) & set(inter.corners()))[0]
        pivot = [pt for pt in inter.corners() if linf(pt, anchor) == 3][0]
        for corner in vol.corners():
            if corner not in set(inter.corners()) and linf(pivot, corner) == 3:
                subvolumes.append(Volume(corners_to_limits(pivot, corner)))
    elif case == 4:
        anchor = list(set(vol.corners()) - set(inter.corners()))[0]
        pivot = [pt for pt in set(inter.corners()) - set(vol.corners()) if linf(pt, anchor) == 3][0] 
        subvolumes.append(Volume(corners_to_limits(pivot, anchor)))
    return subvolumes

def corners_to_limits(c1, c2):
    x0 = min(c1[0], c2[0])
    x1 = max(c1[0], c2[0])
    y0 = min(c1[1], c2[1])
    y1 = max(c1[1], c2[1])
    z0 = min(c1[2], c2[2])
    z1 = max(c1[2], c2[2])
    return [(x0, x1), (y0, y1), (z0, z1)]

def run(queue):
    volumes = [queue.popleft()[1]]

    while queue:
        cmd, current = queue.popleft()

        new_volumes = []

        init_volume = sum(v.volume() for v in volumes)

        inter_volumes = []

        for vol in volumes:
            inter = vol.intersect(current)

            if inter is None or (cmd == 'on' and vol.includes(current)):
                new_volumes.append(vol)
            else:
                inter_volumes.append(inter.volume())
                splitted = split(vol, inter)
                new_volumes.extend(splitted)

        if cmd == 'on':
            new_volumes.append(current)

        volumes = new_volumes.copy()
    return volumes

In [ ]:
queue = read_input(lines)
volumes = run(queue)

In [ ]:
answer_a = sum(v.volume() for v in volumes)

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
def read_input(lines, limit_scale=True):
    queue = deque()

    for cmd, coords in lines:
        x, y, z = coords.split(',')
        l = (x[2:].split('..'), y[2:].split('..'), z[2:].split('..'))
        limits = [(l[0][0], int(l[0][1])+1), (l[1][0], int(l[1][1])+1), (l[2][0], int(l[2][1])+1)]
        queue.append((cmd, Volume(limits)))
            
    return queue

In [ ]:
queue = read_input(lines)
volumes = run(queue)

In [ ]:
answer_b = sum(v.volume() for v in volumes)

In [ ]:
puzzle.answer_b = answer_b

## Day 21
https://adventofcode.com/2021/day/21
### Part 1

In [ ]:
puzzle = Puzzle(2021, 21)

In [ ]:
p01, p02 = [int(line[-1]) for line in puzzle.input_data.split('\n')]

In [ ]:
class Dice:
    
    def __init__(self):
        self.dice = 0
        self.count = 0
        
    def roll(self):
        self.dice = self.dice % 100 + 1
        self.count += 1
        return self.dice
    
class Player:
    def __init__(self, p0):
        self.pos = p0
        self.score = 0

    def move(self, amount):
        self.pos = (self.pos + amount - 1) % 10 + 1
        self.score += self.pos
        
    def __repr__(self):
        return f"{self.pos} {self.score}"

In [ ]:
dice = Dice()

p1, p2 = Player(p01), Player(p02)

win = False
while not win:
    for p in [p1, p2]:
        p.move(sum(dice.roll() for i in range(3)))
        if p.score >= 1000:
            win = True
            break

In [ ]:
answer_a = dice.count * min(p.score for p in [p1, p2])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
from itertools import product
from collections import Counter
table = Counter([sum(p) for p in product([1, 2, 3], repeat=3)])

def play(p1, p2, second=False):
    n1, n2 = 0, 0
    for delta, weight in table.items():
        pos, score = p2 if second else p1
        new_pos = ((pos+delta - 1) % 10) + 1
        score += new_pos
        if score >= 21:
            n1, n2 = (n1, n2+weight) if second else (n1+weight, n2)
        else:
            np1, np2 = (p1, (new_pos, score)) if second else ((new_pos, score), p2)
            sn1, sn2 = play(np1, np2, not(second))
            n1 += (weight*sn1)
            n2 += (weight*sn2)
    return n1, n2

In [ ]:
n1, n2 = play((p01, 0), (p02, 0))

In [ ]:
answer_b = max(n1, n2)

In [ ]:
puzzle.answer_b = answer_b

## Day 20
https://adventofcode.com/2021/day/20
### Part 1

In [ ]:
puzzle = Puzzle(2021, 20)

In [ ]:
algo, input_ = puzzle.input_data.split('\n\n')

In [ ]:
from collections import defaultdict

def get_neighbor(x0, y0, carte):
    string = ''
    for y in range(y0-1, y0+2):
        for x in range(x0-1, x0+2):
            string = f"{string}{carte[x][y]}"
    return int(string, 2)

def parse_input(img):

    carte = defaultdict(lambda : defaultdict( lambda : 0))

    for y, line in enumerate(img.split('\n')):
        for x, c in enumerate(line):
            if c == '#':
                carte[x][y] = 1
                
    return carte

In [ ]:
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (20,20)

import numpy as np

def tonumpy(carte):
    ymin, ymax = min(carte.keys()), max(carte.keys())
    xmin, xmax = min([min(line.keys()) for line in carte.values()]), max([max(line.keys()) for line in carte.values()])
    
    img = np.array([[carte[x][y] for x in range(xmin-2, xmax+2)] for y in range(ymin-2, ymax+2)]).astype('int')
    return img

def expand(carte, default_value):
    new_carte = defaultdict(lambda : defaultdict( lambda : default_value))

    ymin, ymax = min(carte.keys()), max(carte.keys())
    xmin, xmax = min([min(line.keys()) for line in carte.values()]), max([max(line.keys()) for line in carte.values()])

    for y in range(ymin-3, ymax+3):
        for x in range(xmin-3, xmax+3):
            val = get_neighbor(x, y, carte)
            new_carte[x][y] = 1 if algo[val] == '#' else 0
            
    return new_carte

In [ ]:
carte = parse_input(input_)
for i in range(2):
    carte = expand(carte, (i+1) % 2)

In [ ]:
answer_a = np.sum(tonumpy(carte))

In [ ]:
answer_a

In [ ]:
plt.imshow(tonumpy(carte))

### Part 2

In [ ]:
carte = parse_input(input_)
for i in range(50):
    carte = expand(carte, (i+1) % 2)

In [ ]:
answer_b = np.sum(tonumpy(carte))

In [ ]:
puzzle.answer_b = answer_b

In [ ]:
plt.imshow(tonumpy(carte)[100:320,100:320])

## Day 19
https://adventofcode.com/2021/day/19
### Part 1

In [ ]:
puzzle = Puzzle(2021, 19)

In [ ]:
import numpy as np

def parse_input(data):
    scanners = data.split('\n\n')
    scanners = [scanner.split('\n')[1:] for scanner in scanners]
    return [ np.array([[int(v) for v in coords.split(',') ] for coords in scan]) for scan in scanners]

In [ ]:
scanners = parse_input(puzzle.input_data)

In [ ]:
from itertools import combinations

from scipy.spatial.transform import Rotation as R

elems = []

for axis in ['x', 'y', 'z']:
    elems.append([R.from_euler(axis, deg, degrees=True).as_matrix().astype(int) for deg in [0, 90, 180, 270]])
        
rotations = []

for rx in elems[0]:
    for ry in elems[1]:
        for rz in elems[2]:
            rot = np.matmul(np.matmul(rz, ry), rx)
            if not any(np.all(rot == mat) for mat in rotations):
                rotations.append(rot)

In [ ]:
ref = scanners[0]

from collections import Counter

def to_set(array):
    return set(tuple(coords) for coords in array)

def find_delta(pts_src, pts_dst):
    
    targets = to_set(pts_dst)
    
    for src in pts_src:
        for dst in pts_dst:
            delta = src - dst
            if len(set(tuple(coords) for coords in pts_src - delta) & targets) >= 12:
                return True, delta
    return False, None

def get_rotations(scan, rotations):
    return [scan.dot(rot) for rot in rotations]

def find_match(src, dst):

    for scan in get_rotations(src, rotations):
        found, delta = find_delta(scan, dst)
        if found:
            return scan - delta, delta
        
    return None, None
    
points = scanners[0]
assigned = {0}
scanners_pos = {0: (0, 0, 0)}

while len(assigned) != len(scanners):

    for idx, scan in enumerate(scanners):
        if idx in assigned:
            continue
        match, delta = find_match(scan, points)
        if match is not None:
            print(f'Success for {idx}')
            scanners_pos[idx] = delta
            assigned.add(idx)
            points = np.array(list(to_set(points) | to_set(match)))
            break

In [ ]:
answer_a = len(points)

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
answer_b = max([np.sum(np.abs(np.array(pt1) - np.array(pt2))) for pt2 in scanners_pos.values() for pt1 in scanners_pos.values() ])

In [ ]:
puzzle.answer_b = answer_b

## Day 18
https://adventofcode.com/2021/day/18
### Part 1

In [ ]:
puzzle = Puzzle(2021, 18)

In [ ]:
lines = puzzle.input_data.split('\n')

In [ ]:
def parse_tree(string, parent=None):
    if '[' in string:
        return Node(string, parent)
    return Leaf(int(string), parent)

def split(line):
    depth = 0
    split_idx = 0
    for idx, c in enumerate(line):
        if c == '[':
            depth += 1
        elif c == ']':
            depth -= 1
        elif c == ',' and depth == 0:
            split_idx = idx
    return line[:split_idx], line[split_idx+1:]

def join(left, right):
     return f"[{left},{right}]"

class Leaf:

    def __init__(self, val, parent):
        self.val = val
        self.parent = parent
        
    def __repr__(self):
        return str(self.val)
    
    def depth(self):
        return 0
    
    def walk_left(self, origin):
        return self
    
    def walk_right(self, origin):
        return self
    
    def magnitude(self):
        return self.val
    
class Node:
    def __init__(self, _input, parent=None):
        left, right = split(_input[1:-1])
        self.left = parse_tree(left, self)
        self.right = parse_tree(right, self)
        self.parent = parent
        self.val = 0
        
    def __repr__(self):
        return f"[{self.left},{self.right}]"
    
    def is_root(self):
        return self.parent is None
    
    def depth(self):
        return max(self.left.depth(), self.right.depth()) + 1
    
    def height(self):
        if self.is_root():
            return 0
        else:
            return self.parent.height() + 1
        
    def walk_left(self, origin):
        if origin == self.right:
            return self.left.walk_left(self)
        elif origin == self.left:
            if self.is_root():
                return None
            return self.parent.walk_left(self)
        elif origin == self.parent:
            return self.right.walk_left(self)
        
    def walk_right(self, origin):
        if origin == self.left:
            return self.right.walk_right(self)
        elif origin == self.right:
            if self.is_root():
                return None
            return self.parent.walk_right(self)
        elif origin == self.parent:
            return self.left.walk_right(self)
        
    def is_explodable(self):
        return self.height() > 3 and isinstance(self.left, Leaf) and isinstance(self.right, Leaf)
    
    def explode_leftmost(self):
        
        changed = False
        
        for side in ['left', 'right']:
            child = getattr(self, side)
            if isinstance(child, Node):
                if child.is_explodable():
                    left_neighbor = self.walk_left(child)
                    if left_neighbor:
                        left_neighbor.val += child.left.val

                    right_neighbor = self.walk_right(child)
                    if right_neighbor:
                        right_neighbor.val += child.right.val

                    setattr(self, side, Leaf(0, self))

                    return True
                else:
                    ret = child.explode_leftmost()
                    if ret:
                        return True
        return False
    
    def split_leftmost(self):
        
        changed = False
        
        for side in ['left', 'right']:
            child = getattr(self, side)
            if isinstance(child, Leaf):
                if child.val >= 10:
                    left_val = child.val // 2
                    right_val = child.val - left_val
                    setattr(self, side, Node(join(left_val, right_val), self))
                    
                    return True
            else:
                if not child.is_explodable():
                    ret = child.split_leftmost()
                    if ret:
                        return True
        return False
    
    def reduce(self, return_self=True):
        finished = False
        while not finished:
            count = 0
            while self.explode_leftmost():
                count += 1
            if self.split_leftmost():
                count += 1
            finished = count == 0
        if return_self:
            return self
            
    def magnitude(self):
        return 3*self.left.magnitude() + 2*self.right.magnitude()

In [ ]:
def solve(lines):
    current = lines[0]

    for line in lines[1:]:
        tree = parse_tree(join(current, line))
        tree.reduce()
        current = str(tree)
        
    return tree

In [ ]:
solution = solve(lines)

In [ ]:
answer_a = solution.magnitude()

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
from itertools import permutations

pairs = permutations(lines, 2)

In [ ]:
answer_b = max(parse_tree(join(l1, l2)).reduce().magnitude() for l1, l2 in pairs)

In [ ]:
puzzle.answer_b = answer_b

## Day 17
https://adventofcode.com/2021/day/17
### Part 1

In [ ]:
puzzle = Puzzle(2021, 17)
limits = [(coords.split('..')) for coords in puzzle.input_data.replace('target area: ', '').split(', ')]
limits = [(int(l[0][2:]), int(l[1])) for l in limits]

In [ ]:
def launch(vx, vy):
    traj = []
    x, y = 0, 0
    while x <= limits[0][1] and y >= limits[1][0]:
        traj.append((x, y))
        x += vx
        y += vy
        if vx > 0:
            vx = int(vx / abs(vx))*max(abs(vx) - 1, 0)
        vy -= 1

    return traj

def valid(traj):
    return any((limits[0][0] <= x <= limits[0][1]) and (limits[1][0] <= y <= limits[1][1]) for x, y in traj)

t = launch(100, -15)

In [ ]:
ymax = 0
for vx in range(18, 320):
    for vy in range(-10, 500):
        t = launch(vx, vy)
        if valid(t):
            ymax = max(ymax, max(p[1] for p in t))

In [ ]:
answer_a = ymax

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
count = 0
for vx in range(18, 320):
    for vy in range(-80, 1000):
        t = launch(vx, vy)
        if valid(t):
            count += 1

In [ ]:
answer_b = count

In [ ]:
puzzle.answer_b = answer_b

## Day 16
https://adventofcode.com/2021/day/16
### Part 1

In [ ]:
puzzle = Puzzle(2021, 16)
codes = puzzle.input_data

In [ ]:
codes = bin(int(codes, 16))[2:].zfill(len(codes)*4)

In [ ]:
def get_version(stream):
    return int(stream[:3], 2), stream[3:]

def get_type(stream):
    return int(stream[:3], 2), stream[3:]

def get_type_len(stream):
    return int(stream[0], 2), stream[1:]

def parse_literal(stream):
    cont = True
    bits = ''
    payload_len = 6
    while cont:
        cont = bool(int(stream[0]))
        bits += stream[1:5]
        stream = stream[5:]
        payload_len += 5
    return int(bits, 2), stream
    discard = (((payload_len // 4) + 1) * 4) - payload_len


def parse_code(stream):
    ver, stream = get_version(stream)
    type_id, stream = get_type(stream)
    
    data = []
    
    if type_id != 4:
        
        data.append((ver, type_id, -1))
        
        type_len, stream = get_type_len(stream)
        
        if type_len == 0:
            len_subpacket = int(stream[:15], 2)
            stream = stream[15:]
            
            substream = stream[:len_subpacket]
            while substream:
                subdata, substream = parse_code(substream)
                data.extend(subdata)
            stream = stream[len_subpacket:]
        else:
            nb_subpacket = int(stream[:11], 2)
            stream = stream[11:]
            
            for i in range(nb_subpacket):
                subdata, stream = parse_code(stream)
                data.extend(subdata)
            
    else:
        val, stream = parse_literal(stream)
        data.append((ver, type_id, val))
    return data, stream

In [ ]:
data, _ = parse_code(codes)

In [ ]:
answer_a = sum(d[0] for d in data)

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
import numpy as np

def compute(type_id, data):
    if type_id == 0:
        return np.sum(data)
    elif type_id == 1:
        return np.prod(data)
    elif type_id == 2:
        return np.min(data)
    elif type_id == 3:
        return np.max(data)
    elif type_id == 5:
        return int(data[0] > data[1])
    elif type_id == 6:
        return int(data[0] < data[1])
    elif type_id == 7:
        return int(data[0] == data[1])
    
def parse_code_p2(stream):
    ver, stream = get_version(stream)
    type_id, stream = get_type(stream)
    
    if type_id != 4:
        
        data = []
        
        type_len, stream = get_type_len(stream)
        
        if type_len == 0:
            len_subpacket = int(stream[:15], 2)
            stream = stream[15:]
            
            substream = stream[:len_subpacket]
            while substream:
                val, substream = parse_code_p2(substream)
                data.append(val)
            stream = stream[len_subpacket:]
        else:
            nb_subpacket = int(stream[:11], 2)
            stream = stream[11:]
            
            for i in range(nb_subpacket):
                val, stream = parse_code_p2(stream)
                data.append(val)
                
        return compute(type_id, data), stream 
    else:
        val, stream = parse_literal(stream)
        return val, stream

In [ ]:
answer_b, _ = parse_code_p2(codes)

In [ ]:
puzzle.answer_b = answer_b

## Day 15
https://adventofcode.com/2021/day/15

### Part 1

In [ ]:
puzzle = Puzzle(2021, 15)

In [ ]:
import numpy as np

data = np.array([[int(c) for c in line] for line in puzzle.input_data.split('\n')]).astype('uint8')

In [ ]:
from collections import deque

distances = np.zeros(data.shape)
distances[:,:] = 10**20
distances[0, 0] = 0
visited = set()
to_visit = {(0, 0)}
lattice = set((x, y) for y in range(data.shape[0]) for x in range(data.shape[1]))

In [ ]:
def evaluate(node):
    x, y = node
    for dx, dy in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
        neighbor = x+dx, y+dy
        if neighbor in lattice:
            distances[neighbor] = min(distances[node] + data[neighbor], distances[neighbor])
            if neighbor not in visited:
                to_visit.add(neighbor)
    visited.add(node)


In [ ]:
while to_visit:
    node = min([(x, y, distances[x, y]) for x, y in to_visit], key=lambda t: t[2])[:2]
    evaluate(node)
    to_visit = to_visit - visited

In [ ]:
answer_a = int(distances[-1,-1])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
cols = [data]

for i in range(4):
    cols.append((cols[-1] % 9) + 1)

lines = [np.concatenate(cols, axis=1)]

for i in range(4):
    lines.append((lines[-1] % 9) + 1)
data = np.concatenate(lines)

In [ ]:
distances = np.zeros(data.shape)
distances[:,:] = 10**20
distances[0, 0] = 0
visited = set()
to_visit = {(0, 0)}
lattice = set((x, y) for y in range(data.shape[0]) for x in range(data.shape[1]))

In [ ]:
while to_visit:
    node = min([(x, y, distances[x, y]) for x, y in to_visit], key=lambda t: t[2])[:2]
    evaluate(node)
    to_visit = to_visit - visited

In [ ]:
answer_b = int(distances[-1, -1])

In [ ]:
puzzle.answer_b = answer_b

## Day 14
https://adventofcode.com/2021/day/14
### Part 1

In [ ]:
puzzle = Puzzle(2021, 14)

In [ ]:
base, reactions = puzzle.input_data.split('\n\n')
reactions = [reac.split(' -> ') for reac in reactions.split('\n')]
reactions = {r[0]: r[1] for r in reactions}

In [ ]:
def insert(base, reactions):
    insertions = [reactions[base[i:i+2]] for i in range(len(base) - 1)]
    result = "".join(c1+c2 for c1, c2 in zip(base[:-1], insertions))
    result += base[-1]
    return result

In [ ]:
polymer = f"{base}"
for i in range(10):
    polymer = insert(polymer, reactions)

In [ ]:
from collections import Counter

count = Counter(polymer)
answer_a = count.most_common()[0][1] - count.most_common()[-1][1] 

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
def react(pairs, reactions):
    new_count = Counter()
    for pair, count in pairs.items():
        insert = reactions[pair]
        p1, p2 = pair[0]+insert, insert+pair[1]
        new_count[p1] += count
        new_count[p2] += count
    return new_count

In [ ]:
pairs = Counter(base[i:i+2] for i in range(len(base) - 1))

for i in range(40):
    pairs = react(pairs, reactions)

In [ ]:
def count_chars(pairs):
    counter = Counter()
    for pair, count in pairs.items():
        counter[pair[0]] += count
        counter[pair[1]] += count
    return counter

In [ ]:
count = count_chars(pairs)

In [ ]:
answer_b = ((count.most_common()[0][1] - count.most_common()[-1][1])) // 2 + 1

In [ ]:
puzzle.answer_b = answer_b

## Day 13
https://adventofcode.com/2021/day/13
### Part 1

In [ ]:
puzzle = Puzzle(2021, 13)

In [ ]:
coords, folds = puzzle.input_data.split('\n\n')

In [ ]:
xc = [int(c.split(',')[0]) for c in coords.split('\n')]
yc = [int(c.split(',')[1]) for c in coords.split('\n')]

In [ ]:
folding = []
for fold in folds.split('\n'):
    ax, coord = fold.split('=')
    axis = 0 if ax[-1] == 'x' else 1
    folding.append((axis, int(coord)))

In [ ]:
import numpy as np

width, height = max(xc)+1, max(yc)+1
paper = np.zeros((width, height))
paper[xc, yc] = 1

In [ ]:
def fold(paper, folding):
    axis, coord = folding
    if axis == 0:
        first, second = paper[:coord, :], paper[coord+1:, :]
    else:
        first, second = paper[:, :coord], paper[:, coord+1:]
    
    if first.shape != second.shape:
        sx, sy = second.shape
        padded = np.zeros(first.shape)
        padded[:sx, :sy] = second
        second = padded
    return np.maximum(first, np.flip(second, axis))

answer_a = int(np.sum(fold(paper, folding[0])))

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
folded = paper.copy()
for instruction in folding:
    folded = fold(folded, instruction)

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(folded.T)

## Day 12
https://adventofcode.com/2021/day/12
### Part 1

In [ ]:
puzzle = Puzzle(2021, 12)

In [ ]:
edges = [edge.split('-') for edge in puzzle.input_data.split('\n')]

In [ ]:
from collections import defaultdict

graph = defaultdict(set)
for a, b in edges:
    graph[a].add(b)
    graph[b].add(a)

big_caves = set([node for node in graph.keys()if str.lower(node) != node])
nodes = set(graph.keys())

In [ ]:
start = 'start'
end = 'end'

def recurse(path):
    
    current = path[-1]
    
    all_paths = []
    
    for neighbor in graph[current]:
        if neighbor not in path or neighbor in big_caves:
            new_path = path.copy()
            new_path.append(neighbor)
            if neighbor == end:
                all_paths.append(new_path)
            else:
                all_paths.extend(recurse(new_path))
    return all_paths
    
all_paths = recurse([start])
answer_a = len(all_paths)

In [ ]:
puzzle.answer_a = answer_a

### Part2

In [ ]:
small_caves = nodes - big_caves - {start, end}

In [ ]:
from collections import Counter

start = 'start'
end = 'end'

def recurse(path):
    
    current = path[-1]
    
    all_paths = []
    
    for neighbor in graph[current]:
        count = Counter(path)
        if (neighbor not in path) or (neighbor in big_caves) or (neighbor in small_caves and max(count[node] for node in small_caves)== 1):
            new_path = path.copy()
            new_path.append(neighbor)
            if neighbor == end:
                all_paths.append(new_path)
            else:
                all_paths.extend(recurse(new_path))
    return all_paths
    
all_paths = recurse([start])
answer_b = len(all_paths)

In [ ]:
puzzle.answer_b = answer_b

## Day 11
https://adventofcode.com/2021/day/11
### Part 1

In [ ]:
puzzle = Puzzle(2021, 11)

In [ ]:
octo = np.array([[int(c) for c in line] for line in puzzle.input_data.split('\n')]).astype('uint8')

In [ ]:
from itertools import product

import numpy as np

class Nest:
    def __init__(self, octo):
        self.width, self.height = octo.shape
        self.area = octo.copy()
        
    def step(self):
        self.area += 1

        exploded = set()

        while True:
            x, y = np.where(self.area >= 10)

            if x.size == 0:
                break

            self.area[x, y] = 0

            exploded |= set(zip(x, y))

            expanded = [(x+dx,y+dy) for dx, dy in product([-1, 0, 1], repeat=2)]
            nx, ny = np.concatenate([n[0] for n in expanded]), np.concatenate([n[1] for n in expanded])
            valid = (nx >= 0) & (nx < self.width) & (ny >= 0) & (ny < self.height)

            for pos, inc in Counter(zip(nx[valid], ny[valid])).items():
                if pos not in exploded:
                    self.area[pos] += inc

        return len(exploded)
    
    def __repr__(self):
        return "\n".join(["".join([str(self.area[y, x]) for x in range(self.width)]) for y in range(self.height)])



In [ ]:
nest = Nest(octo)
answer_a = sum(nest.step() for i in range(100))

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
nest = Nest(octo)

count = 0
while True:
    n_flashed = nest.step()
    count += 1
    if n_flashed == nest.area.size:
        break
answer_b = count

In [ ]:
puzzle.answer_b = answer_b

## For fun

In [ ]:
import cv2 as cv
import seaborn as sns
import numpy as np

def draw(frame):
    ax = sns.heatmap(frame, vmin=0, vmax=10, cbar=False)
    fig = ax.figure
    
    fig.canvas.draw()

    img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    img = img.reshape(fig.canvas.get_width_height()[::-1] + (3, ))
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
    return img

class FunNest:
    def __init__(self, octo):
        self.width, self.height = octo.shape
        self.area = octo.copy()
        
    def step(self):
        self.area += 1

        exploded = set()
        
        frames = [self.area.copy()]

        while True:
            x, y = np.where(self.area >= 10)

            if x.size == 0:
                break

            self.area[x, y] = 0

            exploded |= set(zip(x, y))

            expanded = [(x+dx,y+dy) for dx, dy in product([-1, 0, 1], repeat=2)]
            nx, ny = np.concatenate([n[0] for n in expanded]), np.concatenate([n[1] for n in expanded])
            valid = (nx >= 0) & (nx < self.width) & (ny >= 0) & (ny < self.height)

            for pos, inc in Counter(zip(nx[valid], ny[valid])).items():
                if pos not in exploded:
                    self.area[pos] += inc
                    
            frames.append(self.area.copy())

        return frames

In [ ]:
max_steps = answer_b + 60
FPS = 25

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv.VideoWriter('media/day11.mp4', fourcc, FPS, (432, 288))

In [ ]:
nest = FunNest(octo)

all_frames = []
for i in range(max_steps):
    all_frames.append(nest.step())

for curr_frames, next_frames in zip(all_frames[:-1], all_frames[1:]):

    curr, next_ = curr_frames[0], next_frames[0]
    
    if len(curr) == 1:
        interp_frames = np.linspace(curr, next_, FPS)

        for frame in interp_frames:
            img = draw(frame)
            writer.write(img)
    else:
        for frame in curr_frames:
            img = draw(frame)
            writer.write(img)
    

writer.release()       

## Day 10
https://adventofcode.com/2021/day/10
### Part 1

In [ ]:
puzzle = Puzzle(2021, 10)

In [ ]:
lines = puzzle.input_data.split('\n')

In [ ]:
from collections import deque

line = lines[1]

mapping = {
    '[': ']',
    '{': '}',
    '(': ')',
    '<': '>',
}

corrupted_scoring = {
    ')': 3,
    ']': 57,
    '}': 1197,
    '>': 25137,
}

def check_line(line):
    stack = deque()
    for char in line:
        if char in mapping:
            stack.append(char)
        else:
            top = stack.pop()
            if mapping[top] != char:
                return corrupted_scoring[char]
    return 0

answer_a = sum(check_line(line) for line in lines)

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
incomplete_lines = [line for line in lines if check_line(line) == 0]

In [ ]:
repair_scoring = {
    ')': 1,
    ']': 2,
    '}': 3,
    '>': 4,
}

def repair_line(line):
    stack = deque()
    for char in line:
        if char in mapping:
            stack.append(char)
        else:
            top = stack.pop()
    
    score = 0
    while stack:
        top = stack.pop()
        score *= 5
        score += repair_scoring[mapping[top]]
    return score

scores = sorted(repair_line(line) for line in incomplete_lines)
answer_b = scores[len(scores) // 2]

In [ ]:
puzzle.answer_b = answer_b

## Day 9
https://adventofcode.com/2021/day/9
### Part 1

In [ ]:
puzzle = Puzzle(2021, 9)

!pip install opencv-python matplotlib

In [ ]:
import cv2 as cv

depth = np.array([[int(c) for c in line] for line in puzzle.input_data.split('\n')]).astype('uint8')

In [ ]:
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (10,10)

plt.imshow(depth)

In [ ]:
erosion = cv.erode(depth, cv.getStructuringElement(cv.MORPH_CROSS, (3, 3)), borderType=cv.BORDER_REFLECT)

delta = depth - erosion
minima = np.logical_and(delta == 0, depth != 9)

answer_a = np.sum(depth[np.where(minima)] + 1)

In [ ]:
puzzle.answer_a = answer_a

### Quick and dirty

In [ ]:
import numpy as np

from collections import defaultdict, Counter

depth = defaultdict(lambda : defaultdict(lambda : 10))

for y, line in enumerate(puzzle.input_data.split('\n')):
    for x, char in enumerate(line):
        depth[x][y] = int(char)

In [ ]:
W, H = len(depth), len(depth[0])
minima = []
for x in range(W):
    for y in range(H):
        val = depth[x][y]
        if val < depth[x-1][y] and val < depth[x][y-1] and val < depth[x+1][y] and val < depth[x][y+1]:
            minima.append((x, y))

answer_a = sum(depth[p[0]][p[1]]+1 for p in minima)

### Part 2

In [ ]:
from collections import deque

class Bassin:
    
    def __init__(self, minimum):
        self.points = {minimum}
    
    def expand(self):
        
        queue = deque(list(self.points))
        
        while queue:
            x, y = queue.popleft()
            
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                pos = (x+dx, y+dy)
                if pos in self.points:
                    continue
                if depth[pos[0]][pos[1]] < 9 and depth[pos[0]][pos[1]] >= depth[x][y]:
                    queue.append(pos)
                    self.points.add(pos)
    def size(self):
        return len(self.points)

In [ ]:
bassins = [Bassin(minimum) for minimum in minima]
for bassin in bassins:
    bassin.expand()


In [ ]:
import math

answer_b = math.prod(sorted([bassin.size() for bassin in bassins], reverse=True)[:3])

In [ ]:
puzzle.answer_b = answer_b

## Day 8
https://adventofcode.com/2021/day/8
### Part 1

In [ ]:
puzzle = Puzzle(2021, 8)

In [ ]:
lines = [line.split('|') for line in puzzle.input_data.split('\n')]
lines = [(s1.split(), s2.split()) for s1, s2 in lines]

In [ ]:
from collections import Counter

count = Counter()

for _, output in lines:
    for digit in output:
        count[len(digit)] += 1
        
answer_a = count[2] + count[3] + count[4] + count[7]

In [ ]:
puzzle.answer_a = answer_a

### Part2

In [ ]:
# I dont have time for this shit solution

def outsmart_problem(patterns):
    
    voc = {}
    pos = {}
    
    voc[1] = [el for el in patterns if len(el) == 2][0]
    voc[7] = [el for el in patterns if len(el) == 3][0]
    voc[4] = [el for el in patterns if len(el) == 4][0]
    voc[8] = [el for el in patterns if len(el) == 7][0]
    pos[0] = set(voc[7]) - set(voc[1])
    voc[9] = [el for el in patterns if len(el) == 6 and len(set(el) - set(voc[4])) == 2][0]
    pos[6] = set(voc[9]) - set(voc[4]) - pos[0]
    voc[0] = [el for el in patterns if len(el) == 6 if el != voc[9] and set(voc[7]) <= set(el)][0]
    voc[6] = [el for el in patterns if len(el) == 6 if el != voc[9] and el != voc[0]][0]
    pos[3] = set(voc[6]) - set(voc[0])
    pos[2] = set(voc[0]) - set(voc[6])
    voc[5] = [el for el in patterns if len(el) == 5 and not (pos[2] <= set(el))][0]
    pos[5] = set(voc[1]) - pos[2]
    voc[3] = [el for el in patterns if len(el) == 5 and el != voc[5] and pos[5] <= set(el)][0]
    voc[2] = [el for el in patterns if len(el) == 5 and el != voc[5] and not(pos[5] <= set(el))][0]
    
    code_map = {frozenset(v): k for k, v in voc.items()}
    
    return code_map

def decode_line(line):
    patterns, display = line
    
    code_map = outsmart_problem(patterns)
    
    nb = 0
    for signal in display:
        nb = nb*10 + code_map[frozenset(signal)]
        
    return nb

In [ ]:
puzzle.answer_b = answer_b

## Day 7
https://adventofcode.com/2021/day/7
### Part 1

In [ ]:
puzzle = Puzzle(2021, 7)

In [ ]:
pos = [int(p) for p in puzzle.input_data.split(',')]

In [ ]:
import numpy as np

mu = round(np.mean(pos))
answer_a = min([sum([abs(mu + delta - p) for p in pos]) for delta in range(-1000, 1000)])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
def cost(start, end):
    dist = abs(start - end)
    return (dist * (dist + 1)) / 2
    
answer_b = round(min([sum([cost(mu + delta, p) for p in pos]) for delta in range(-1000, 1000)]))

In [ ]:
puzzle.answer_b = answer_b

## Day 6
https://adventofcode.com/2021/day/6
### Part 1

In [ ]:
puzzle = Puzzle(2021, 6)

In [ ]:
numbers = puzzle.input_data.split(',')

In [ ]:
from collections import Counter

def initialize(numbers):
    count = Counter()

    for nb in numbers:
        count[int(nb)] += 1
        
    return count

In [ ]:
max_age = 9

def simulate(count, nb_days):
    for day in range(nb_days):
        new_count = Counter()
        for age in range(max_age):
            new_count[(age-1)%max_age] = count[age]
        new_count[6] += count[0]
        count = new_count
    return count

In [ ]:
answer_a = sum(simulate(initialize(numbers), 80).values())

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
answer_b = sum(simulate(initialize(numbers), 256).values())

In [ ]:
puzzle.answer_b = answer_b

## Day 5
https://adventofcode.com/2021/day/5
### Part 1

In [ ]:
puzzle = Puzzle(2021, 5)

In [ ]:
class Line:
    
    def __init__(self, p0, p1):
        self.x0 = int(p0[0])
        self.y0 = int(p0[1])
        self.x1 = int(p1[0])
        self.y1 = int(p1[1])
        
        # not pleased with linf but l2 gives overlapping points due to discretization
        self.len = max(abs(self.x1 - self.x0), abs(self.y1 - self.y0))

        self.dx = (self.x1 - self.x0) / self.len
        self.dy = (self.y1 - self.y0) / self.len
        
    def is_horizontal(self):
        return self.y0 == self.y1
    
    def is_vertical(self):
        return self.x0 == self.x1
    
    def __iter__(self):
        self.__iter_ptr = 0
        return self
        
    def __next__(self):
        if self.__iter_ptr <= self.len:
            vx = (self.__iter_ptr * self.dx) + self.x0
            vy = (self.__iter_ptr * self.dy) + self.y0
            self.__iter_ptr += 1
            return (round(vx), round(vy))
        else:
            raise StopIteration
            
from collections import defaultdict, Counter

def create_map(lines):

    vents = defaultdict(Counter)

    for line in lines:
        for pt in line:
            vents[pt[0]][pt[1]] += 1
    
    return vents

def create_line(line):
    p1, p2 = line.split(' -> ')
    return Line(p1.split(','), p2.split(','))

raw_lines = puzzle.input_data.split('\n')
lines = list(map(create_line, raw_lines))

In [ ]:
non_diag = list(filter(lambda l : l.is_horizontal() or l.is_vertical(), lines))
vents = create_map(non_diag)
answer_a = len([val for col in vents.values() for val in col.values() if val > 1])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
vents = create_map(lines)
answer_b = len([val for col in vents.values() for val in col.values() if val > 1])

In [ ]:
puzzle.answer_b = answer_b

### For fun

In [ ]:
# For fun
!pip install seaborn

In [ ]:
import seaborn as sns
import numpy as np

from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (15,15)

def visualize(vents):
    width = max(vents.keys()) + 10
    height = max(max(col) for col in vents.values()) + 10

    data = np.array([[vents[x][y] for y in range(height)] for x in range(width)])
    
    sns.heatmap(data)

In [ ]:
visualize(create_map(non_diag))

In [ ]:
visualize(create_map(lines))

## Day 4
https://adventofcode.com/2021/day/4
### Part 1

In [ ]:
!pip install numpy

In [ ]:
puzzle = Puzzle(2021, 4)

In [ ]:
data = puzzle.input_data.split('\n\n')
numbers, boards = data[0], data[1:]
numbers = [int(nb) for nb in numbers.split(',')]
boards = list(map(lambda b: [int(loc.strip()) for loc in b.split()], boards))

In [ ]:
import numpy as np

class Board:
    def __init__(self, brd):
        rows = np.array(brd).reshape(5, -1)
        cols = rows.T

        self.rows = [set(row) for row in rows]
        self.cols = [set(col) for col in cols]
        
    def play(self, number):
        self.rows = [set(row) - {number} for row in self.rows]
        self.cols = [set(col) - {number} for col in self.cols]
        
    def has_won(self):
        return any([not any(row) for row in self.rows]) or any([not any(col) for col in self.cols])
    
    def remaining_sum(self):
        return sum(sum(row) for row in self.rows)

In [ ]:
def play(board, numbers):
    brd = Board(board)
    for idx, nb in enumerate(numbers):
        brd.play(nb)
        if brd.has_won():
            break
    return idx, brd

games = sorted([play(board, numbers) for board in boards], key=lambda t: t[0])

In [ ]:
idx, board = games[0]
answer_a = numbers[idx]*board.remaining_sum()

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
idx, board = games[-1]
answer_b = numbers[idx]*board.remaining_sum()

In [ ]:
puzzle.answer_b = answer_b

## Day 3
https://adventofcode.com/2021/day/3
### Part 1

In [ ]:
puzzle = Puzzle(2021, 3)

commands = puzzle.input_data.split('\n')

In [ ]:
from collections import Counter

acc = Counter()
for cmd in commands:
    acc += Counter({idx: int(val) for idx, val in enumerate(cmd)})

In [ ]:
N, K = len(commands), len(commands[0])
gamma = '0b'+''.join(str(int(acc[i] > N/2)) for i in range(K))
gamma = int(gamma, 2)
epsilon = (~gamma & 0xFFF)
answer_a = gamma*epsilon

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
def filter_cmds(commands, pos, lcb):
    acc = sum(int(cmd[pos]) for cmd in commands)
    mc = (acc >= len(commands) / 2)
    if lcb:
        mc = not mc
    mc = int(mc)
    return [cmd for cmd in commands if int(cmd[pos]) == mc]

def apply(cmds, lcb=False):
    for pos in range(K):
        cmds = filter_cmds(cmds, pos, lcb)
        if len(cmds) == 1:
            break
    return cmds

oxy = int(apply(commands.copy(), lcb=False)[0], 2)
co2 = int(apply(commands.copy(), lcb=True)[0], 2)

answer_b = oxy*co2

In [ ]:
puzzle.answer_b = answer_b

## Day 2
https://adventofcode.com/2021/day/2
### Part 1

In [ ]:
from collections import Counter

puzzle = Puzzle(2021, 2)

commands = puzzle.input_data.split('\n')

global_cmd = Counter()

for cmd in commands:
    word, amount = cmd.split()
    global_cmd[word] += int(amount)
    
answer_a = global_cmd['forward']*(global_cmd['down'] - global_cmd['up'])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
status = Counter()

for cmd in commands:
    word, amount = cmd.split()
    amount = int(amount)
    if word == "forward":
        status['forward'] += amount
        status['depth'] += amount*status['aim']
    elif word == "up":
        status['aim'] -= amount
    else:
        status['aim'] += amount
        
answer_b = status["forward"]*status["depth"]

In [ ]:
puzzle.answer_b = answer_b

## Day 1
https://adventofcode.com/2021/day/1
### Part 1

In [ ]:
puzzle = Puzzle(2021, 1)

depths = [int(depth) for depth in puzzle.input_data.split()]
answer_a = len([d1 for d1, d2 in zip(depths[:-1], depths[1:]) if d2 > d1])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
moving_average = [d1+d2+d3 for d1, d2, d3 in zip(depths[:-2], depths[1:-1], depths[2:])]
answer_b = len([d1 for d1, d2 in zip(moving_average[:-1], moving_average[1:]) if d2 > d1])

In [ ]:
puzzle.answer_b = answer_b